In [1]:
import pandas as pd

# Define the two DataFrames
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 1], ['d', 2]],
                   columns=['letter', 'number'])

# Concatenate the DataFrames vertically (axis=0) and reset the index
# The default axis is 0, so we don't need to specify it.
df_combined = pd.concat([df1, df2], ignore_index=True)

# Print the resulting DataFrame and its index type
print("Combined DataFrame:")
print(df_combined)
print("\nIndex Type:")
print(df_combined.index)

Combined DataFrame:
  letter  number
0      a       1
1      b       2
2      c       1
3      d       2

Index Type:
RangeIndex(start=0, stop=4, step=1)


In [1]:
import pandas as pd

# Define the first DataFrame (df1)
df1_dict = {
        'id': ['1', '2', '3', '4', '5'],
        'Feature1': ['A', 'C', 'E', 'G', 'I'],
        'Feature2': ['B', 'D', 'F', 'H', 'J']}
df1 = pd.DataFrame(df1_dict, columns = ['id', 'Feature1', 'Feature2'])

# Define the second DataFrame (df2)
df2_dict = {
        'id': ['1', '2', '6', '7', '8'],
        'Feature1': ['K', 'M', 'O', 'Q', 'S'],
        'Feature2': ['L', 'N', 'P', 'R', 'T']}
df2 = pd.DataFrame(df2_dict, columns = ['id', 'Feature1', 'Feature2'])

print("--- DataFrame 1 ---")
print(df1)
print("\n--- DataFrame 2 ---")
print(df2)
print("\n" + "="*50 + "\n")

--- DataFrame 1 ---
  id Feature1 Feature2
0  1        A        B
1  2        C        D
2  3        E        F
3  4        G        H
4  5        I        J

--- DataFrame 2 ---
  id Feature1 Feature2
0  1        K        L
1  2        M        N
2  6        O        P
3  7        Q        R
4  8        S        T




In [2]:
# Inner Merge: Keeps only the intersecting IDs ('1' and '2')
df_inner = pd.merge(df1, df2, on='id', how='inner')

print("Output 1: Inner Merge")
print(df_inner)

Output 1: Inner Merge
  id Feature1_x Feature2_x Feature1_y Feature2_y
0  1          A          B          K          L
1  2          C          D          M          N


In [4]:
# Outer Merge: Keeps all IDs from both DataFrames
df_outer = pd.merge(
    df1,
    df2,
    on='id',
    how='outer',
    suffixes=('_df1', '_df2') # Set custom suffixes for clarity
)

# Replace 'nan' with 'nannan' for visual match with your example (optional step)
df_outer = df_outer.fillna('nan')

print("\nOutput 2: Full Outer Merge")
print(df_outer)


Output 2: Full Outer Merge
  id Feature1_df1 Feature2_df1 Feature1_df2 Feature2_df2
0  1            A            B            K            L
1  2            C            D            M            N
2  3            E            F          nan          nan
3  4            G            H          nan          nan
4  5            I            J          nan          nan
5  6          nan          nan            O            P
6  7          nan          nan            Q            R
7  8          nan          nan            S            T
